In [ ]:
# En esta celda se puede probar la funcion de analisis de una imagen

from lib.image import image_data
from lib.analysis import dyssynchrony_analysis
import cv2

# Path completo donde esta la imagen (incluyendo nombre y extencion)
path = '/media/leandro/20d94040-1ffc-4389-948c-746fd67f15fe/Lean/Analizador_imagenes_calcio/230819sel/230819Dan2/'
photo_name = 'D2d000'
photo = photo_name + '.tif'
# Ancho de la feta a analisar (es un parametro opcional)
slice_width = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 30
# Calibracion del tiempo de cada pixel
calibration = 4.5

image = cv2.imread(path + photo_name + '.tif')
# Select ROI
fromCenter = False
showCrosshair = False
seleted_parameters = cv2.selectROI(image, fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

image = image_data.get_image_data(image)
image = image_data.crop_vertical(image, x_start, x_end)
image = image_data.crop_horizontal(image, y_start, y_end)

results = dyssynchrony_analysis.analyze_image(image, min_dist_between_maxs, calibration, slice_width=slice_width)

In [ ]:
jasonGenerator(path,results,photo_name)

In [ ]:
slicesParser_amplitudes(path)

In [ ]:
print(results['image']['max_peaks_positions'])
for i in range(0,len(results['slices'])):
    array = results['slices'][i]['max_peaks_positions']
    print(i,array)

In [ ]:
import matplotlib.pyplot as plt

for i in range(0,len(results['slices'])):
    array = results['slices'][i]['intensities']
    plt.plot(array) # plotting by columns
    plt.show()

# PRUEBA ANALISIS 

In [ ]:
# En esta celda se puede probar la funcion de analisis todas las imagenes .tif en un directorio
import os
from lib.image import image_data
from lib.analysis import dyssynchrony_analysis
import cv2

# Ancho de la feta a analisar (es un parametro opcional)
slice_width = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 200

# Calibracion del tiempo de cada pixel
calibration = 3.1

# Path donde estan las imagenes
path = "C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/1 Hz/"

images_paths = [ f'{path}/{file}' for file in os.listdir(path) if file.endswith(".tif") ]
images = [ cv2.imread(image_path) for image_path in images_paths ]

In [ ]:
fromCenter = False
showCrosshair = False

#Tomamos la primer imagen para seleccionar el recorte elegido el resto de las imagenes se van a recortar igual
seleted_parameters = cv2.selectROI(images[0], fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

In [ ]:
images = [ image_data.get_image_data(image) for image in images ]
images = [ image_data.crop_vertical(image, x_start, x_end) for image in images ]
images = [ image_data.crop_horizontal(image, y_start, y_end) for image in images ]

In [ ]:
results = [(dyssynchrony_analysis.analyze_image(image, min_dist_between_maxs, calibration, slice_width=5), print ('Done')) for image in images]

# Analisis de datos
Notas: 
- Image corresponde al análisis de toda la célula y slices corresponde a fetas de esa imagen
- intensidades es el valor que resulta de comprimir (sumando) la matriz con las coordenadas de x,y,z de los pixeles 
- max_peaks_pos es la index de en la lista de intensidades 
- max_peaks_intensities el valor de intensidad que se corresponde con el pico en la posicion analoga de max_peaks_pos

In [1]:
#Imports
import json
import numpy as np
import pandas as pd

In [2]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results,photo_name):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result_' + photo_name + '.json',"w") as miarch:
        miarch.write(json.dumps(results, cls=NumpyEncoder))

In [3]:
def jasonReader(path, photo):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open( path + 'analysis_result_' + photo+ '.json','r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [4]:
def slicesParser_max_peaksI(path):
#     read the analysis result 
    dictres = jasonReader(path,photo_name)

#extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x) for x in range(0,len(allSlices[0]['max_peaks_intensities']))]    
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        for i in range(0,len(allSlices)):
            df_sum.loc[i] = allSlices[i]['max_peaks_intensities'][0:] 
        return df_sum

    except:
        print('no possible callculation')

In [5]:
def slicesParser_halfpeakstime(path):
    #read the analysis result 
    dictres = jasonReader(path,photo_name)
    #extract all the slices data
    allSlices = dictres['slices']
#     try:
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_half_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_half_peaks']
    return df_sum

#     except:
#         print('no possible callculation')

In [6]:
def slicesParser_amplitudes(path):
    #read the analysis result 
    dictres = jasonReader(path,photo_name)
    #extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['amplitudes']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['amplitudes']

        return df_sum
    
    except:
        print('no possible callculation')

In [7]:
def slicesParser_min_peaksI(path):
    #read the analysis result 
    dictres = jasonReader(path,photo_name)
    #extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['min_peaks_intensities']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['min_peaks_intensities']

        return df_sum
    
    except:
        print('no possible callculation')

In [8]:
def slicesParser_peaktime(path):
    #read the analysis result 
    dictres = jasonReader(path,photo_name)
    #extract all the slices data
    allSlices = dictres['slices']

    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_peaks']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['times_to_peaks']

        return df_sum
    
    except:
        print('no possible callculation')

In [9]:
def slicesParser_tau(path):
    #read the analysis result 
    dictres = jasonReader(path,photo_name)
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    try:
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['tau_s']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['tau_s']
        
        return df_sum
    
    except:
        print('no possible callculation')

In [10]:
def BasicStasts(df):
    if df.empty == False:
        return df.astype('int').describe()

In [11]:
def discordIndex(df):
    return df.iloc[2].mean()

In [12]:
def amplitudes_ratio(df):
    df_alt = pd.DataFrame()
    for i in range(0,len(list(df))-1):
        name = 'ratio' + str(i+1) +'-' + str(i)
        df_alt[name] = df[list(df)[i+1]]/df[list(df)[i]]
    return df_alt

In [ ]:
jasonGenerator(path,results,photo_name)

In [ ]:
# path = '/home/leandro/Documentos/Analisis_de_imagenes/CardiAP/DesktopApp/photos_examples/'
photo_name = 'c1d000'
photo = photo_name + '.tif'

In [ ]:
loaded_dict = jasonReader(path,photo_name)

In [ ]:
BasicStasts(slicesParser_max_peaksI(path))

In [ ]:
amplitudes_ratio(slicesParser_amplitudes(path)).std()

In [ ]:
amp_mean = slicesParser_amplitudes(path).mean()
for i in range(0,len(amp_mean)-1):
    AR_ind = (1- (amp_mean[i+1])/amp_mean[i])
    print (AR_ind)

In [ ]:
slicesParser_min_peaksI(path).head(2)

In [ ]:
slicesParser_tau(path).head()

In [13]:
def DI(path, photo_name):
    DIs = pd.DataFrame()
    for j in range (1, len(slicesParser_amplitudes(path).columns)):
        rel_diff = []
        for i in range (0, len (slicesParser_amplitudes(path))):
            T1 = list(slicesParser_amplitudes(path)['transient'+str(j)])[i]
            T2 = list(slicesParser_amplitudes(path)['transient'+str(j+1)])[i]
            rel_diff.append((T1-T2)/max(T1,T2))
        DIs[j] = rel_diff
    return DIs
def AR(path):
    ARs = []
    for j in range (1, len(slicesParser_amplitudes(path).columns)):
        T1_mean = float(slicesParser_amplitudes(path)['transient'+str(j)].mean())
        T2_mean = float(slicesParser_amplitudes(path)['transient'+str(j+1)].mean())
        ARs.append((T1_mean-T2_mean)/T1_mean)
    return ARs

In [14]:
### Extract jsons from files and analyse ###

import glob, os
paths = [x[0] for x in os.walk('/media/leandro/20d94040-1ffc-4389-948c-746fd67f15fe/Lean/Analizador_imagenes_calcio/AR_analysis/')]
for path in paths:
    try:
        pathes = [x[1] for x in os.walk(path)]
        for pathe in pathes:
            for item in pathe:
#                 photo_name_list.append(item)
                print (item)
        path = path + '/'
        print(path, DI(path).std())
    except FileNotFoundError:
        pass

NameError: name 'photo_name' is not defined

In [ ]:
    os.chdir(path)
    for file in glob.glob("*.json"):
        print(file)

In [ ]:
DI.std().mean()

In [ ]:
AR_mean = sum(AR)/len(AR) 

In [ ]:
AR_mean

In [17]:
photo_name_list = []
pathes = [x[1] for x in os.walk('/media/leandro/20d94040-1ffc-4389-948c-746fd67f15fe/Lean/Analizador_imagenes_calcio/AR_analysis/')]
for pathe in pathes:
    for item in pathe:
        photo_name_list.append(item)

In [18]:
photo_name_list

['031019Sel',
 '121218SEL',
 '071119SEL',
 '170719SEL',
 '050320sel',
 '010719SEL',
 '060320sel',
 '311019sel',
 '220119sel',
 '230819sel',
 '010819Sel',
 '171019sel',
 '070119Sel',
 '031019c2',
 '031019c3',
 '031019c5',
 '031019c4',
 '031019c1',
 '071119vk2',
 '071119dan3',
 '071119c1sel',
 '071119dan4',
 '071119vk5',
 '170719VK7',
 '170719VK4',
 '170719Dan3',
 '170719Dan6',
 '170719c1',
 '170719Dan8',
 '170719c5',
 '050320c6',
 '050320c1',
 '050320c7',
 '050320c2',
 '050320c8',
 '050320c4',
 '050320c5',
 '050320c3',
 '010719ctrol1',
 '010719VK3',
 '010719VK7',
 '010719Dan2',
 '010719Dan6',
 '010719ctrol8',
 '060320dan7',
 '060320vk2',
 '060320dan6',
 '060320c1',
 '060320vk3',
 '060320vk5',
 '060320dan4',
 '311019vk4',
 '311019dan5',
 '311019vk2',
 '311019c6sel',
 '311019c1sel',
 '311019dan3',
 '220119VK1uM',
 '220119c1',
 '230819Dan2',
 '230819VK2',
 '230819c1',
 '230819Dan1',
 '230819VK1',
 '230819c2',
 '010819c3',
 '010819c1',
 '010819c2',
 '171019dan4',
 '171019dan7',
 '171019c1se